In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings

import pandas as pd
import numpy as np

from src.data import etl
from src.data import bq

# Setup
warnings.filterwarnings('ignore')

In [3]:
# Diretório fonte dos dados
dir_fonte='../../../data/interim/inmet_historico'

# Pastas das subdivisões por ano
pastas = os.listdir(dir_fonte)

In [ ]:
# Para cada pasta
for sub_pasta in pastas:
    arquivos = os.listdir(dir_fonte + '/' + sub_pasta)
    for arq in arquivos:
        print(arq)
        # Lendo apenas as especificações do arquivo
        df_info = pd.read_csv(
            dir_fonte + '/' + sub_pasta + '/' +arq, 
            sep=';', 
            encoding='ANSI', 
            decimal=',',
            nrows=7,
            names=['ATRIBUTO', 'VALOR'],
            header=None,
        )

        # Lendo dados
        df = pd.read_csv(
            dir_fonte + '/' + sub_pasta + '/' +arq, 
            sep=';', 
            encoding='ANSI',
            decimal=',',
            skiprows=8 
        )

        df = etl.processa_csv_inmet(df_info, df)
        df = etl.transformacao_inmet(df)
        
        table_id = 'alert-being-435923-n9.inmet_historico.dados_historico'
        bq.exporta_dados(df, table_id)
        